In [1]:
# Variable
W_max = 19.5;
W_min = 10;
import xlrd
import xlwt
import os


## excel parameter
# parameter schedule end Index
Schedule_endIndex = 289
# student schedule end Index
EndTime = 39;

# function for formating
def timeFormating(num):
    n = str(num)
    time = n.split(".")
    if time[1]=='5':
        time[1]='50' 
    min=str(int(time[1])*60/100).split(".")[0]
    if min =='0':
        min='00'
    if int(time[0]) <12:
        return time[0]+":"+min+"am"
    elif int(time[0])==12:
        return time[0]+":"+min+"pm"
    elif int(time[0])> 24:
        return str(int(time[0])-24)+":"+min+"am"
    elif int(time[0]) >12:
        return str(int(time[0])-12)+":"+min+"pm"
    
def dayFormating(num):
    if num==1:
        return "Mon"
    elif num ==2:
        return "Tue"
    elif num==3:
        return "Wed"
    elif num==4:
        return "Thu"
    elif num==5:
        return "Fri"
    elif num==6:
        return "Sat"
    elif num==7:
        return "Sun"

loc = ("schedule.xls")
wb = xlrd.open_workbook(loc)
sheet = wb.sheet_by_index(0)
x = list(range(1, Schedule_endIndex))
#initial data
startTime = []
processStartTime = []
endTime = []
shiftslength = []
days = []
shiftstations = []
station = []

# get shift data
for i in x:
    timeStarted = sheet.cell_value(i,1)
    startTime.append(timeStarted)
    if str(timeStarted)[-3:] ==".25" or str(timeStarted)[-3:] ==".75" :
        processStartTime.append(timeStarted-0.25)
    else :
        processStartTime.append(timeStarted)  
    endTime.append(sheet.cell_value(i,2))
    shiftslength.append(sheet.cell_value(i,3))
    days.append(sheet.cell_value(i,4))
    shiftstations.append(sheet.cell_value(i,5))
    if sheet.cell_value(i,5) not in station:
        station.append(sheet.cell_value(i,5))

shifts = list(range(0, Schedule_endIndex-1))
# get all students availability and prefer time
studentName = []
p = []
#allAvailability[i]:availability for student i
#allAvailability[i][j]:availability for student i on day j
#allAvailability[i][j][k]:availability for student i on day j at time k
allAvailability = []
dirs = os.listdir("studentAvailability")
for file in dirs:
    if (not file.startswith("~$")) and (not file.startswith(".")) and (not file.startswith("_")):
        studentName.append(file.split(".")[0])
        # get each student availability
        wbp=xlrd.open_workbook("studentAvailability/"+file)
        sheetperson = wbp.sheet_by_index(0)
        # get prefer working time
        p.append(sheetperson.cell_value(1,8))
        time_slot = list(range(1, EndTime))
        stuAvailability = []
        for j in list(range(1,8)):
            day_slot = []
            for i in time_slot:
                day_slot.append(sheetperson.cell_value(i,j))
            stuAvailability.append(day_slot)
        allAvailability.append(stuAvailability)
# get whether shift is available for each student
a=[]
for i in range(len(studentName)):
    stuShiftAvailability = []
    stuAvailability = allAvailability[i]
    #loop all shifts
    for j in shifts:
        shiftstartTime = startTime[j]
        shiftLength = shiftslength[j]
        day = int(days[j]-1)
        startTimeSlot = int((shiftstartTime-7)/0.5)
        lengthSlot = int(shiftLength/0.5)
        # get student availability for the specific day
        stuDayAvailability = stuAvailability[day]
        # loop one single shift
        available = 1;
        for k in list(range(startTimeSlot,startTimeSlot+lengthSlot)):
            if(stuDayAvailability[k] ==0):
                available=0
                break
        stuShiftAvailability.append(available)
    a.append(stuShiftAvailability)

notavailable = []
for i in shifts:
    for j in range(i, len(shifts)):
        if days[i] == days[j] and i !=j:
            if startTime[i] <= startTime[j] and endTime[i] > startTime[j]:
                notavailable.append([i,j])
            elif endTime[j] > startTime[i] and endTime[j] <= endTime[i]:
                notavailable.append([i,j])

In [2]:
del allAvailability
# model building
from gurobipy import *
import statistics
m = Model()
# initial variable
#x[i][j] assign student i to shift j
x=[]
for i in range(len(studentName)):
    y = [None]*len(shifts)
    x.append(y)
    for j in shifts:
        x[i][j] = m.addVar(vtype=GRB.BINARY)
m.update()

# add constrain
for i in range(len(studentName)):
    expr = LinExpr()
    for j in shifts:
        if a[i][j] ==0:
            m.addConstr(x[i][j]==0)
        #expr.add(x[i][j],shiftslength[j])
    expr.addTerms(shiftslength,[x[i][j] for j in shifts])
    m.addConstr(expr<=W_max)
    m.addConstr(expr>=W_min)

    for k in notavailable:
        m.addConstr(x[i][k[0]]+x[i][k[1]] <= 1)

for j in shifts:
    expr = LinExpr()
    expr.addTerms([1]*len(studentName),[x[i][j] for i in range(len(studentName))])
    #
    #for i in range(len(studentName)):
    #    expr.add(x[i][j],1)
    m.addConstr(expr<=1)

# goal
# F1
F1 = 0
F2 = 0
for i in range(len(studentName)):
    scheduleTime = 0
    stuStation = []
    unique = []
    lateNightshift = 0
    for j in shifts:
        if x[i][j] == 1:
            scheduleTime += x[i][j]*shiftslength[j]
            stuStation.append(shiftstations[j])
            if shiftstations[j] not in unique :
                unique.append(shiftstations[j])
            
            if shiftstations[j] == "Late Night":
                lateNightshift+=1
                
    F1 += (scheduleTime-p[i])*(scheduleTime-p[i])
    F2 += len(stuStation)-lateNightshift-len(unique)
# Define the objective function
ohm_1 = 3
ohm_2 = 9
m.Params.MIPGapAbs =2.5
m.Params.MIPFocus=3
m.Params.ImproveStartTime = 600
m.setObjective((ohm_1*F1+ ohm_2*F2)/len(studentName), GRB.MINIMIZE)
m.optimize()

Academic license - for non-commercial use only
Changed value of parameter MIPGapAbs to 2.5
   Prev: 1e-10  Min: 0.0  Max: 1e+100  Default: 1e-10
Changed value of parameter MIPFocus to 3
   Prev: 0  Min: 0  Max: 3  Default: 0
Changed value of parameter ImproveStartTime to 600.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Optimize a model with 224901 rows, 26496 columns and 514321 nonzeros
Model has 3828672 quadratic objective terms
Variable types: 0 continuous, 26496 integer (26496 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+00]
  Objective range  [1e+00, 5e+00]
  QObjective range [1e-01, 3e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Presolve removed 220170 rows and 14025 columns
Presolve time: 1.49s
Presolved: 4731 rows, 12471 columns, 82402 nonzeros
Presolved model has 905987 quadratic objective terms
Variable types: 0 continuous, 12471 integer (12471 binary)
Presolve removed 86 rows and 0 columns
Presolved: 4645 rows, 12557 c

In [3]:
workbook = xlwt.Workbook()
sheet = workbook.add_sheet('schedule')
sheet.write(0,0,"Shift")
sheet.write(0,1,"Time")
sheet.write(0,2,"Station")
for k, value in enumerate(shifts):
    sheet.write(int(k)+1, 0, k+1)
    sheet.write(int(k)+1, 1, dayFormating(days[k])+":"+timeFormating(startTime[k])+"-"+timeFormating(endTime[k]))
    sheet.write(int(k)+1, 2, shiftstations[k])
for j in range(0,len(studentName)):
    sheet.write(0,3+j,studentName[j])
    f = open("schedule/"+studentName[j]+".txt", "w")
    f.write(studentName[j]+":\n")
    writeMessage = ''
    totalHour = 0
    for i in shifts:
        if (type(x[j][i]) != int) and (x[j][i].x==1) :
            totalHour += x[j][i].x*shiftslength[i]
            sheet.write(i+1,3+j,"Assign")
            writeMessage += dayFormating(days[i])+": "+timeFormating(startTime[i])+"-"+\
                            timeFormating(endTime[i])+", ("+str(shiftstations[i])+")\n"
    f.write('Total hour: '+str(totalHour)+'\n')
    f.write(writeMessage)
    f.close()
workbook.save('assignSchedule.xls')
#check objective function 1
print("Result for objective function 1")
difference = 0
print("Student: Total time:\tPrefer:\tDifference")
for j in range(0,len(studentName)):
    writeMessage = ''
    totalstuHour = 0
    for i in shifts:
        if (type(x[j][i]) != int) and (x[j][i].x==1) :
            totalstuHour += x[j][i].x*shiftslength[i]
    difference+=abs(totalstuHour-p[j])
    print(studentName[j]+"\t "+str(totalstuHour)+"\t\t"+str(p[j])+"\t"+str(abs(totalstuHour-p[j])))
print("Total difference is "+str(difference))
print()
print()
# check objective function 2
print("Result for objective function 2")
totalStation = 0
totalunique = 0
print("Student: No. shifts:\tNo. different station:\tDifference")
for i in range(len(studentName)):
    stuStation = []
    unique = []
    for j in shifts:
        if x[i][j].x == 1:
            scheduleTime += x[i][j].x*shiftslength[j]
            stuStation.append(shiftstations[j])
            if shiftstations[j] not in unique :
                unique.append(shiftstations[j])
    totalStation+=len(stuStation)
    totalunique+=len(unique)
    print(studentName[i]+"\t "+str(len(stuStation))+"\t\t"+str(len(unique))+"\t\t\t"+str(len(stuStation)- len(unique)))
print("Total\t "+str(totalStation)+"\t\t"+str(totalunique)+"\t\t\t"+str(totalStation-totalunique))

F1 = 0
F2 = 0
for i in range(len(studentName)):
    scheduleTime = 0
    stuStation = []
    unique = []
    for j in shifts:
        if x[i][j].x == 1:
            scheduleTime += x[i][j].x*shiftslength[j]
            stuStation.append(shiftstations[j])
            if shiftstations[j] not in unique :
                unique.append(shiftstations[j])
    F1 += (scheduleTime-p[i])*(scheduleTime-p[i])
    F2 += len(stuStation)-len(unique)
print()
print(F1)
print(F2)
ohm_1 = 3
ohm_2 = 9
print((ohm_1*F1+ ohm_2*F2)/len(studentName))

Result for objective function 1
Student: Total time:	Prefer:	Difference
11	 10.25		13.0	2.75
39	 13.0		15.5	2.5
38	 10.5		14.0	3.5
10	 13.0		15.0	2.0
12	 13.25		15.0	1.75
13	 10.5		10.0	0.5
17	 10.5		12.0	1.5
16	 10.5		14.0	3.5
28	 10.5		12.0	1.5
14	 10.5		13.5	3.0
15	 14.75		16.0	1.25
29	 11.25		15.0	3.75
72	 10.5		11.0	0.5
66	 11.0		12.0	1.0
8	 11.0		12.0	1.0
9	 14.5		17.0	2.5
67	 10.0		11.5	1.5
73	 10.25		10.0	0.25
65	 10.0		11.0	1.0
71	 10.0		11.0	1.0
59	 10.5		14.0	3.5
58	 10.0		11.5	1.5
70	 10.25		12.0	1.75
64	 10.25		10.0	0.25
48	 10.0		10.5	0.5
60	 10.25		13.0	2.75
74	 10.5		13.0	2.5
75	 10.25		10.0	0.25
61	 10.25		12.5	2.25
49	 10.0		13.5	3.5
88	 13.25		16.0	2.75
77	 10.75		12.0	1.25
63	 12.0		13.0	1.0
62	 10.0		11.0	1.0
76	 10.25		11.0	0.75
89	 13.25		15.5	2.25
90	 12.25		14.5	2.25
84	 10.0		13.0	3.0
1	 10.0		10.0	0.0
53	 10.75		11.0	0.25
47	 10.5		10.0	0.5
46	 11.75		14.5	2.75
52	 11.0		14.0	3.0
85	 10.5		10.0	0.5
91	 10.5		13.5	3.0
87	 10.0		12.0	2.0
44	 10.0		11.5	1.5
50	 

In [4]:
# print remaining shifts
print("Shift\tStart Time\tEnd Time\tLength\tDay\tStation")
n = 0
for j in shifts:
    assign = 0
    for i in range(len(studentName)):
        if x[i][j].x==1:
            assign=1
            break
    if assign == 0:
        n+=1
        print(j,"\t",timeFormating(startTime[j]),"\t",timeFormating(endTime[j]),"\t",shiftslength[j],"\t",dayFormating(days[j]),"\t",shiftstations[j])
print("Total number of unassigned shifts: ",n)

Shift	Start Time	End Time	Length	Day	Station
Total number of unassigned shifts:  0
